In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kushagra3204/wheat-plant-diseases")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/wheat-plant-diseases


In [16]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.optimizers import Adam 
from sklearn.metrics import classification_report

In [5]:
BASE_PATH = '/kaggle/input/wheat-plant-diseases/wheat-plant-diseases/'

DATA_DIR_TRAIN = '/kaggle/input/wheat-plant-diseases/data/train'
DATA_DIR_TEST = '/kaggle/input/wheat-plant-diseases/data/test'
IMAGE_SIZE = (299, 299) 
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 5

In [6]:
TRAIN_CLASS_FOLDERS = ['Aphid', 'Blast', 'Mildew', 'Smut', 'Tan spot']
TEST_CLASS_FOLDERS = ['aphid_test', 'blast_test', 'mildew_test', 'smut_test', 'tan_spot_test']
REPORT_TARGET_CLASSES = ['Aphid', 'Wheat Blast', 'Powdery Mildew', 'Smut', 'Spot Blotch']

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# TRAINING GENERATOR
print("--- Initializing Data Generators ---")
try:
    train_generator = train_datagen.flow_from_directory(
        DATA_DIR_TRAIN, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TRAIN_CLASS_FOLDERS, class_mode='categorical', subset='training', shuffle=True
    )
    # VALIDATION GENERATOR
    validation_generator = train_datagen.flow_from_directory(
        DATA_DIR_TRAIN, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TRAIN_CLASS_FOLDERS, class_mode='categorical', subset='validation', shuffle=False
    )
    # TEST GENERATOR
    test_generator = test_datagen.flow_from_directory(
        DATA_DIR_TEST, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, 
        classes=TEST_CLASS_FOLDERS, class_mode='categorical', shuffle=False
    )
    
    print(f"Training Samples: {train_generator.n}")
    print(f"Validation Samples: {validation_generator.n}")
    print(f"Test Samples: {test_generator.n}")

except Exception as e:
    print(f"\nERROR: Could not load data generators. Check the paths and folder names.")
    print(f"Details: {e}")
    exit()

--- Initializing Data Generators ---
Found 3770 images belonging to 5 classes.
Found 941 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Training Samples: 3770
Validation Samples: 941
Test Samples: 250


In [9]:
print("\n--- Building and Training Xception Model ---")

# 1. Load Base Model (Xception)
base_model_xception = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
)

total_layers = len(base_model_xception.layers)
UNFREEZE_LAYERS_FROM = total_layers - 40 

for layer in base_model_xception.layers:
    layer.trainable = True 
for layer in base_model_xception.layers[:UNFREEZE_LAYERS_FROM]:
    layer.trainable = False 

x = base_model_xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model_xception.input, outputs=predictions)



--- Building and Training Xception Model ---


I0000 00:00:1765226138.407445      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1765226138.408105      47 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
# 3. Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.00001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 149, 149,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 149, 149,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 147, 147,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 147, 147,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 147, 147,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 147, 147,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 147, 147,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 147, 147,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 147, 147,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 147, 147,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 74, 74,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 74, 74,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 74, 74,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 74, 74,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 74, 74,    │          0 │ add[0][0]       

 Total params: 21,913,133 (83.59 MB)

 Trainable params: 11,605,981 (44.27 MB)

 Non-trainable params: 10,307,152 (39.32 MB)

In [11]:
# 4. Train Model
start_time = time.time()
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.samples // BATCH_SIZE, 
    epochs=EPOCHS, 
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    verbose=1
)
training_time = time.time() - start_time

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1765226154.679541     195 service.cc:148] XLA service 0x7932e80022c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765226154.680671     195 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1765226154.680691     195 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1765226156.447691     195 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-12-08 20:36:02.070558: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng3{k11=0} for conv (f32[32,128,147,147]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,128,147,147]{3,2,1,0}, f32[128,128,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0

 73/117 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.2473 - loss: 1.6067

E0000 00:00:1765226270.142059     194 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765226270.292285     194 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765226271.114470     194 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765226271.254542     194 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765226271.651550     194 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

117/117 ━━━━━━━━━━━━━━━━━━━━ 216s 2s/step - accuracy: 0.2864 - loss: 1.5695 - val_accuracy: 0.4914 - val_loss: 1.3356
Epoch 2/15
  1/117 ━━━━━━━━━━━━━━━━━━━━ 45s 392ms/step - accuracy: 0.5312 - loss: 1.2657

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


117/117 ━━━━━━━━━━━━━━━━━━━━ 25s 211ms/step - accuracy: 0.5312 - loss: 1.2657 - val_accuracy: 0.5097 - val_loss: 1.3254
Epoch 3/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 120s 1s/step - accuracy: 0.5680 - loss: 1.1998 - val_accuracy: 0.6649 - val_loss: 1.0708
Epoch 4/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 25s 208ms/step - accuracy: 0.7500 - loss: 0.8631 - val_accuracy: 0.6681 - val_loss: 1.0706
Epoch 5/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.7083 - loss: 0.9135 - val_accuracy: 0.7263 - val_loss: 0.8741
Epoch 6/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 25s 211ms/step - accuracy: 0.8438 - loss: 0.7143 - val_accuracy: 0.7047 - val_loss: 0.8682
Epoch 7/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.7447 - loss: 0.7530 - val_accuracy: 0.7478 - val_loss: 0.7450
Epoch 8/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 25s 211ms/step - accuracy: 0.7500 - loss: 0.5871 - val_accuracy: 0.7500 - val_loss: 0.7460
Epoch 9/15
117/117 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.7973 - loss: 0.6404 - val_accurac

In [12]:
# 5. Evaluate and Report
print(f"\n--- VGG19 FINAL RESULTS ---")

# Evaluate on Test Generator
loss, accuracy = model.evaluate(test_generator, steps=test_generator.n // BATCH_SIZE + 1, verbose=0)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Training Time: {training_time:.2f} seconds")


--- VGG19 FINAL RESULTS ---
Test Accuracy: 0.8480
Training Time: 1249.02 seconds


In [13]:
# Generate Classification Report
test_generator.reset()
Y_pred = model.predict(test_generator, steps=test_generator.n // BATCH_SIZE + 1, verbose=0)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

report = classification_report(
    y_true, 
    y_pred, 
    target_names=REPORT_TARGET_CLASSES, 
    zero_division=0,
    output_dict=True 
)

print("\n--- CLASSIFICATION REPORT ---")
print(classification_report(y_true, y_pred, target_names=REPORT_TARGET_CLASSES, zero_division=0))


--- CLASSIFICATION REPORT ---
                precision    recall  f1-score   support

         Aphid       0.80      0.86      0.83        50
   Wheat Blast       0.88      1.00      0.93        50
Powdery Mildew       0.95      0.78      0.86        50
          Smut       0.88      0.98      0.92        50
   Spot Blotch       0.74      0.62      0.67        50

      accuracy                           0.85       250
     macro avg       0.85      0.85      0.84       250
  weighted avg       0.85      0.85      0.84       250



In [14]:
model_name = "Xception"

metrics_summary = {
    'Model': model_name,
    'Accuracy': report['accuracy'],
    'Precision': report['weighted avg']['precision'], 
    'Recall': report['weighted avg']['recall'],
    'F1-Score': report['weighted avg']['f1-score'],
    'Training Time (s)': training_time,
}

df_metrics = pd.DataFrame([metrics_summary])


In [15]:
results_file = 'model_performance_summary.csv'
if not os.path.exists(results_file):
    df_metrics.to_csv(results_file, index=False)
else:
    df_metrics.to_csv(results_file, mode='a', header=False, index=False)
    
print(f"\nMetrics saved to {results_file}")


Metrics saved to model_performance_summary.csv
